In [54]:
import pandas as pd

In [55]:
import requests

# Список ключевых слов
keywords = ['JavaScript', 'Angular', 'React', 'Java', 'Python', 'PHP', 'UX/UI',
           'C++', 'C#', 'Go', 'Rails', 'Android', 'IOS', 'Flutter', 'React Native']

# Регион поиска (Москва)
hh_area = 1

# Базовый URL для API hh.ru
base_url = 'https://api.hh.ru/vacancies'

# Параметры запроса
params = {
    'area': hh_area,
    'per_page': 100,  # Максимальное количество вакансий на страницу
    'clusters': 'true'  # Включение кластеров для обхода ограничения на 2000 строк
}

# Функция для выполнения запроса к API
def fetch_vacancies(keyword):
    params['text'] = keyword
    page = 0
    all_vacancies = []

    while True:
        params['page'] = page
        response = requests.get(base_url, params=params)
        data = response.json()

        if 'items' in data:
            all_vacancies.extend(data['items'])
            if data['pages'] - 1 <= page:
                break
        else:
            break

        page += 1

    return all_vacancies

# Получение вакансий для каждого ключевого слова
all_vacancies = []
for keyword in keywords:
    vacancies = fetch_vacancies(keyword)
    all_vacancies.extend(vacancies)

# Вывод количества найденных вакансий
print(f'Найдено вакансий: {len(all_vacancies)}')

# Пример вывода первых 5 вакансий
for vacancy in all_vacancies[:5]:
    print(f"Название: {vacancy['name']}, Компания: {vacancy['employer']['name']}")

Найдено вакансий: 17670
Название: QA-тестировщик /QA engineer (Middle/Senior), Компания: АПТЕКА ВАША № 1
Название: Frontend-разработчик (react), Компания: Кибертех
Название: Специалист (Frontend-разработчик), Компания: Первый канал
Название: Frontend-разработчик, Компания: Brainbox
Название: QA Middle инженер, Компания: Премиум Бонус


In [56]:
df = pd.DataFrame(all_vacancies)

# Удаление ненужных или неинформативных столбцов
columns_to_drop = [
    'brand_snippet', 'branding', 'adv_context', 'is_adv_vacancy',
    'adv_response_url', 'fly_in_fly_out_duration', 'working_time_modes',
    'working_time_intervals', 'working_days', 'contacts', 'relations',
    'url', 'alternate_url', 'insider_interview', 'show_logo_in_search',
    'apply_alternate_url', 'archived', 'sort_point_distance', 'type', 
    'response_url', 'internship', 'night_shifts', 'created_at',
    'department', 'snippet', 'work_format', 'working_hours', 
    'work_schedule_by_days', 'accept_temporary', 'employment_form',
    'accept_incomplete_resumes',
]
df = df.drop(columns=columns_to_drop, errors='ignore')

In [57]:
# Извлечение нужных данных из вложенных словарей
df['salary_from'] = df['salary'].apply(lambda x: x['from'] if isinstance(x, dict) and 'from' in x else None)
df['salary_to'] = df['salary'].apply(lambda x: x['to'] if isinstance(x, dict) and 'to' in x else None)
df['salary_currency'] = df['salary'].apply(lambda x: x['currency'] if isinstance(x, dict) and 'currency' in x else None)
df['address_raw'] = df['address'].apply(lambda x: x['raw'] if isinstance(x, dict) and 'raw' in x else None)
df['employer_name'] = df['employer'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None)
df['employer_accredited_it_employer'] = df['employer'].apply(lambda x: x['accredited_it_employer'] if isinstance(x, dict) and 'accredited_it_employer' in x else None)
df['employer_trusted'] = df['employer'].apply(lambda x: x['trusted'] if isinstance(x, dict) and 'trusted' in x else None)
df['schedule_name'] = df['schedule'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None)
df['professional_roles_name'] = df['professional_roles'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 and 'name' in x[0] else None)
df['experience_name'] = df['experience'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None)
df['employment_name'] = df['employment'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None)

# Удаление старых столбцов
df = df.drop(columns=['salary', 'address', 'employer', 'schedule', 'professional_roles', 'experience', 'employment'])

# Конвертация валют USD и EUR в рубли
exchange_rates = {
    'USD': 110,  # Примерный курс
    'EUR': 105  # Примерный курс
}

def convert_to_rub(row):
    if row['salary_currency'] in exchange_rates:
        rate = exchange_rates[row['salary_currency']]
        if row['salary_from'] is not None:
            row['salary_from'] *= rate
        if row['salary_to'] is not None:
            row['salary_to'] *= rate
    return row


df = df.apply(convert_to_rub, axis=1)
df = df.drop(columns=['salary_currency'])

In [59]:
# Сохранение DataFrame в CSV файл
df.to_csv('vacancies.csv', index=False)


In [60]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17670 entries, 0 to 17669
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               17670 non-null  object 
 1   premium                          17670 non-null  bool   
 2   name                             17670 non-null  object 
 3   has_test                         17670 non-null  bool   
 4   response_letter_required         17670 non-null  bool   
 5   area                             17670 non-null  object 
 6   published_at                     17670 non-null  object 
 7   salary_from                      5670 non-null   float64
 8   salary_to                        4821 non-null   float64
 9   address_raw                      7650 non-null   object 
 10  employer_name                    17670 non-null  object 
 11  employer_accredited_it_employer  17638 non-null  object 
 12  employer_trusted  

In [61]:
df

,id,premium,name,has_test,response_letter_required,area,published_at,salary_from,salary_to,address_raw,employer_name,employer_accredited_it_employer,employer_trusted,schedule_name,professional_roles_name,experience_name,employment_name
0,110240273,False,QA-тестировщик /QA engineer (Middle/Senior),False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-03T12:22:49+0300,80000.0,NaN,"Москва, 3-й проезд Перова Поля, 3А",АПТЕКА ВАША № 1,False,True,Полный день,Тестировщик,От 3 до 6 лет,Полная занятость
1,113850442,False,Frontend-разработчик (react),False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-17T09:48:30+0300,NaN,NaN,None,Кибертех,True,True,Удаленная работа,"Программист, разработчик",От 3 до 6 лет,Полная занятость
2,114053537,False,Специалист (Frontend-разработчик),False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-18T15:44:22+0300,NaN,NaN,"Москва, улица Академика Королёва, 12",Первый канал,False,True,Полный день,"Программист, разработчик",От 1 года до 3 лет,Полная занятость
3,114077445,False,Frontend-разработчик,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-18T18:07:31+0300,200000.0,NaN,None,Brainbox,False,True,Удаленная работа,"Программист, разработчик",От 3 до 6 лет,Полная занятость
4,113854986,False,QA Middle инженер,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-17T10:50:35+0300,NaN,NaN,"Москва, 2-я Синичкина улица, 9А",Премиум Бонус,True,True,Полный день,Тестировщик,От 1 года до 3 лет,Полная занятость
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17665,111773666,False,Ведущий системный аналитик,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-09T11:38:35+0300,NaN,NaN,"Москва, Пресненская набережная, 8с1",Capital Group,False,True,Полный день,Другое,Более 6 лет,Полная занятость
17666,107710891,False,React Native developer (+Node.js),False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-16T12:57:55+0300,NaN,NaN,"Москва, Вольная улица, 35с2",КОНТРОЛ+,True,True,Полный день,"Программист, разработчик",От 3 до 6 лет,Полная занятость
17667,110606113,False,Senior DevOps Engineer,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-16T10:00:59+0300,NaN,NaN,None,Colvir Software Solutions,False,True,Полный день,DevOps-инженер,Более 6 лет,Полная занятость
17668,111351714,False,Frontend-разработчик для Программно-Аппаратног...,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",2024-12-16T17:59:32+0300,NaN,NaN,"Москва, улица Кульнева, 3с1",Anima Technologies,False,True,Полный день,"Программист, разработчик",От 3 до 6 лет,Полная занятость
